In [14]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import  OpenAIEmbeddings
from langchain_chroma import Chroma



In [3]:
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
import pandas as pd

books = pd.read_csv("books_cleaned.csv")

In [45]:
books["tagged_description"]

0       9780002005883 A NOVEL THAT READERS and critics...
1       9780002261982 A new 'Christie for Christmas' -...
2       9780006178736 A memorable, mesmerizing heroine...
3       9780006280897 Lewis' work on the nature of lov...
4       9780006280934 "In The Problem of Pain, C.S. Le...
                              ...                        
5192    9788172235222 On A Train Journey Home To North...
5193    9788173031014 This book tells the tale of a ma...
5194    9788179921623 Wisdom to Create a Life of Passi...
5195    9788185300535 This collection of the timeless ...
5196    9789027712059 Since the three volume edition o...
Name: tagged_description, Length: 5197, dtype: object

In [5]:
books["tagged_description"].to_csv(
    "tagged_description.txt",
    index=False,
    header=False,
    lineterminator="\n"
)

In [6]:
raw_documents = TextLoader("tagged_description.txt").load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100, separator="\n")
documents = text_splitter.split_documents(raw_documents)

Created a chunk of size 1170, which is longer than the specified 1000
Created a chunk of size 1216, which is longer than the specified 1000
Created a chunk of size 1090, which is longer than the specified 1000
Created a chunk of size 1191, which is longer than the specified 1000
Created a chunk of size 1267, which is longer than the specified 1000
Created a chunk of size 2010, which is longer than the specified 1000
Created a chunk of size 1225, which is longer than the specified 1000
Created a chunk of size 1186, which is longer than the specified 1000
Created a chunk of size 1216, which is longer than the specified 1000
Created a chunk of size 1193, which is longer than the specified 1000
Created a chunk of size 1059, which is longer than the specified 1000
Created a chunk of size 1272, which is longer than the specified 1000
Created a chunk of size 1637, which is longer than the specified 1000
Created a chunk of size 1132, which is longer than the specified 1000
Created a chunk of s

In [7]:
documents[0]

Document(metadata={'source': 'tagged_description.txt'}, page_content='"9780002005883 A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, G

In [8]:
db_books = Chroma.from_documents(
    documents,
    embedding=OpenAIEmbeddings())

In [9]:
query = "A book to teach children about nature"
docs = db_books.similarity_search(query, k = 10)
docs

[Document(id='94ef4da7-3fd9-4973-8d99-5d2f9190a9a1', metadata={'source': 'tagged_description.txt'}, page_content='"9780786808069 Children will discover the exciting world of their own backyard in this introduction to familiar animals from cats and dogs to bugs and frogs. The combination of photographs, illustrations, and fun facts make this an accessible and delightful learning experience."\n"9780786808373 Introducing your baby to birds, cats, dogs, and babies through fine art, illsutration and photographs. These books are a rare opportunity to expose little ones to a range of images on a single subject, from simple child\'s drawings and abstract art to playful photos. A brief text accompanies each image, introducing baby to some basic -- and sometimes playful -- information on the subjects."'),
 Document(id='a0d9116a-1b0a-4221-ade4-9f07c8631d05', metadata={'source': 'tagged_description.txt'}, page_content='"9780786808380 Introduce your babies to birds, cats, dogs, and babies through f

In [11]:
raw_isbn = docs[0].page_content.split()[0].strip().strip('"')
isbn = int(raw_isbn)
books[books["isbn13"] == isbn]

isbn13      isbn10                                title  \
3747  9780786808069  0786808063  Baby Einstein: Neighborhood Animals   

                                authors        categories  \
3747  Marilyn Singer;Julie Aigner-Clark  Juvenile Fiction   

                                              thumbnail  \
3747  http://books.google.com/books/content?id=X9a4P...   

                                            description  published_year  \
3747  Children will discover the exciting world of t...          2001.0   

      average_rating  num_pages  ratings_count  \
3747            3.89       16.0          180.0   

                       title_and_subtitle  \
3747  Baby Einstein: Neighborhood Animals   

                                     tagged_description  
3747  9780786808069 Children will discover the excit...

In [12]:
def retrieve_semantic_recommendations(
        query: str,
        top_k: int = 10,
) -> pd.DataFrame:
    recs = db_books.similarity_search(query, k = 50)

    books_list = []

    for i in range(0, len(recs)):
        books_list += [int(recs[i].page_content.split()[0].strip().strip('"'))]


    return books[books["isbn13"].isin(books_list)]

In [13]:
retrieve_semantic_recommendations("A book to teach children about nature")

isbn13      isbn10  \
30    9780006646006  000664600X   
87    9780060001537  0060001534   
223   9780060775858  0060775858   
260   9780060852559  0060852550   
324   9780060959036  0060959037   
399   9780062516374  006251637X   
413   9780064405850  0064405850   
424   9780064410724  0064410722   
442   9780067575208  006757520X   
799   9780142003343  0142003344   
879   9780152045739  0152045732   
1077  9780240806082  0240806085   
1220  9780312315733  0312315732   
1479  9780345434906  0345434900   
1639  9780374422080  0374422087   
1641  9780374517885  0374517886   
1696  9780375713934  037571393X   
1905  9780393311037  0393311031   
2261  9780446518628  044651862X   
2301  9780446677493  0446677493   
2409  9780451456816  0451456815   
2497  9780486256429  0486256421   
2503  9780486276809  0486276805   
2858  9780590032490  0590032496   
2897  9780618111206  0618111204   
3000  9780671631987  0671631985   
3197  9780689707490  0689707495   
3201  9780689823824  0689823827   
3213  9780689860072  0689860072   
3470  9780747566755  0747566755   
3576  9780762408283  0762408286   
3582  9780763621643  0763621641   
3591  9780764122781  0764122789   
3604  9780764584657  0764584650   
3676  9780767908184  076790818X   
3747  9780786808069  0786808063   
3749  9780786808380  0786808381   
3751  9780786808717  0786808713   
3764  9780786818723  0786818727   
3767  9780786849734  0786849738   
3920  9780810117495  0810117495   
4185  9780878937202  087893720X   
4231  9780898599596  0898599598   
4264  9780941807555  094180755X   
4331  9781400033850  1400033853   
4370  9781400064892  1400064899   
4455  9781405201711  1405201711   
4780  9781584850557  1584850558   
4809  9781586857721  158685772X   
5120  9781895176407  1895176409   

                                                  title  \
30                                   Ocean Star Express   
87                                    Diary of a Spider   
223             Goodnight Moon 60th Anniversary Edition   
260                          Animal, Vegetable, Miracle   
324                                     Prodigal Summer   
399                            Rainforest Home Remedies   
413            Strawberry Girl 60th Anniversary Edition   
424                                  Four Stupid Cupids   
442                                 The Sense of Wonder   
799                                     The Blank Slate   
879   The Little Red Lighthouse and the Great Gray B...   
1077                          Directing the Documentary   
1220                                    Little Children   
1479                             The Real James Herriot   
1639                             Everything on a Waffle   
1641                                           Sammlung   
1696                                 John James Audubon   
1905                       Hen's Teeth and Horse's Toes   
2261                             The Celestine Prophecy   
2301                    Hope is the Thing with Feathers   
2409                                  Not of Woman Born   
2497                                          Galapagos   
2503                    The Secret Garden Coloring Book   
2858                                        The witches   
2897                                        Once a Wolf   
3000       Teach Your Child to Read in 100 Easy Lessons   
3197                  Cloudy With a Chance of Meatballs   
3201                         A Child's Garden of Verses   
3213                                       Dragonsinger   
3470                             The Story of Mr Sommer   
3576                         The Mini Zen Gardening Kit   
3582                                       Castle Diary   
3591                                    Aquatic Turtles   
3604                              Astronomy For Dummies   
3676               A Short History of Nearly Everything   
3747                Baby Einstein: Neighborhood Animals   
3749                              Baby Einst